Podklady pro články Anety Martínkové na iROZHLAS.cz v prosinci 2023.

In [1]:
import os
import pandas as pd

In [2]:
from src.tabulka import tabulka
from src.graf import graf

In [3]:
cz = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))

In [4]:
cz_interpreti = cz['interpret_hlavni'].drop_duplicates().to_list()

In [5]:
def nejmensispolecny(x):
    x = x.lower().split('(')[0]
    if x[0:3] == "the":
        x = x.replace("the ","")
    return x

In [6]:
wikid = pd.read_json(os.path.join("wikidata","wikidata.json"))
wikid['interpret_hlavni'] = wikid['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))

## vs. Billboard 100

In [7]:
b100 = pd.read_csv(os.path.join("data_b100","b100.csv"))

In [8]:
b100 = b100[b100['rok'] >= 2006]

In [9]:
us_interpreti = b100['interpret_hlavni'].drop_duplicates().to_list()

In [10]:
interpreti = cz_interpreti + us_interpreti

In [11]:
interpreti_slovnik = {}
for i in interpreti:
    malym = nejmensispolecny(i)
    interpreti_slovnik[malym] = i

In [12]:
cz = cz.sort_values(by = ['rok','tyden','pozice'], ascending=True)

In [13]:
cz['interpret_hlavni'] = cz['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))
cz['track'] = cz['track'].apply(lambda x: nejmensispolecny(x))

In [14]:
cz['wk'] = cz['week'].rank(method='dense').astype(int)

In [15]:
b100['week'] = b100.apply(lambda row: f"""{row['rok']}-{row['tyden']:02}""", axis = 1)

In [16]:
b100 = b100.rename(columns = {'song': 'track'})

In [17]:
b100['interpret_hlavni'] = b100['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))
b100['track'] = b100['track'].apply(lambda x: nejmensispolecny(x))

In [18]:
cz = cz.sort_values(by="week")
b100 = b100.sort_values(by="week")

In [19]:
b100.tail()

,date,rank,track,artist,last-week,peak-rank,weeks-on-board,rok,datum,tyden,interpret_hlavni,week
317414,2021-11-06,28,let's go brandon,Bryson Gray Featuring Tyson James & Chandler C...,NaN,28,1,2021,11-06,45,bryson gray,2021-45
317413,2021-11-06,27,moth to a flame,Swedish House Mafia & The Weeknd,NaN,27,1,2021,11-06,45,swedish house mafia,2021-45
317411,2021-11-06,25,chasing after you,Ryan Hurd With Maren Morris,25.0,23,26,2021,11-06,45,ryan hurd,2021-45
317422,2021-11-06,36,buy dirt,Jordan Davis Featuring Luke Bryan,34.0,34,12,2021,11-06,45,jordan davis,2021-45
317486,2021-11-06,100,nevada,YoungBoy Never Broke Again,NaN,58,4,2021,11-06,45,youngboy never broke again,2021-45


In [20]:
cz = cz[cz['week'] <= '2021-45']

In [21]:
[x for x in b100['week'].drop_duplicates().to_list() if x not in cz['week'].drop_duplicates().to_list()]

['2006-51',
 '2007-52',
 '2009-51',
 '2010-50',
 '2011-26',
 '2011-51',
 '2011-53',
 '2012-52',
 '2013-52',
 '2014-33',
 '2014-51',
 '2015-52',
 '2016-51',
 '2016-53',
 '2017-51',
 '2018-51',
 '2019-30',
 '2020-01']

Pozor, seznamy týdnů se úplně nepřekrývají, srovnání může lítat o týden.

### Porovnání interpretů

In [22]:
am_jednicky = pd.Series(b100[b100['rank'] == 1].groupby('interpret_hlavni')['track'].nunique())
am_jednicky = am_jednicky[am_jednicky >= 3]
am_jednicky = am_jednicky.index.to_list()

In [23]:
am_jednicky

['adele',
 'ariana grande',
 'beyoncé',
 'black eyed peas',
 'britney spears',
 'bruno mars',
 'bts',
 'cardi b',
 'drake',
 'eminem',
 'fergie',
 'flo rida',
 'justin bieber',
 'justin timberlake',
 'katy perry',
 'lady gaga',
 'mariah carey',
 'maroon 5',
 'p!nk',
 'post malone',
 'rihanna',
 'taylor swift',
 'travis scott',
 'weeknd']

In [24]:
americke_jednicky = b100[b100['rank'] == 1]['interpret_hlavni'].drop_duplicates().to_list()

In [25]:
americke_jednicky_v_cesku = pd.Series(cz[cz['interpret_hlavni'].isin(am_jednicky)].groupby('interpret_hlavni')['pozice'].min().reindex(am_jednicky).sort_values())

In [26]:
americke_jednicky_v_cesku = pd.DataFrame(americke_jednicky_v_cesku[americke_jednicky_v_cesku != 1]).reset_index().rename(columns = {'interpret_hlavni':'Interpreti','pozice':'Nejlepší pozice v ČR'}).replace({"Interpreti": interpreti_slovnik})

In [27]:
wikid

,interpret_hlavni,id,stazeno,nazev_wiki,popis_en,popis_cs,velikost,zanry,datum_narozeni,datum_umrti,...,zeme_puvodu,obcanstvi,etnicita,hlas,nastroje,oceneni,nabozenstvi,orientace,spotify_id,spotify_odberatele
0,'n sync,Q154454,True,NSYNC,American boy band,hudební skupina,boy band,[teen pop],NaN,NaN,...,Spojené státy americké,None,None,None,None,"[Umělec roku, Hvězda na Hollywoodském chodníku...",None,None,6Ff53KvcvAj5U7Z1vojB5o,NaN
1,2 chainz,Q129668,True,2 Chainz,American rapper (born 1977),None,člověk,"[Southern hip hop, hip hop, Gangsta rap, trap,...",1977.0,NaN,...,Spojené státy americké,[Spojené státy americké],None,None,"[sampler, bicí souprava, syntezátor, hlas]",[BET Award for Best Collaboration],None,None,17lzZA2AlOHwCwFALHttmp,NaN
2,2 pac,Q112240732,True,2 Packesel,painting by H.v. Zügel,None,malba,None,NaN,NaN,...,Česko,None,None,None,None,None,None,None,None,NaN
3,21 savage,Q25095399,True,21 Savage,English-born rapper,None,člověk,"[hip hop, trap, Gangsta rap, Horrorcore]",1992.0,NaN,...,Spojené království,[Spojené království],[Black British],None,[hlas],None,"[Ifá, ateismus]",None,1URnnhqYAYcrqrcwql10ft,NaN
4,24kgoldn,Q83751918,True,24kGoldn,American rapper and singer,None,člověk,[pop rap],2000.0,NaN,...,Spojené státy americké,[Spojené státy americké],None,None,[hlas],[Billboard 21 Under 21],None,None,6fWVd57NKTalqvmjRd2t8Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,čechomor,Q2424041,True,Čechomor,Czech traditional music band,česká hudební skupina,hudební skupina,"[lidová hudba, lidová píseň]",NaN,NaN,...,Česko,None,None,None,None,[Cena Anděl Skupina roku],None,None,None,NaN
1599,české srdce,Q42709632,True,České srdce,None,česká hudební skupina,hudební skupina,[rock],NaN,NaN,...,Česko,None,None,None,None,None,None,None,None,NaN
1600,škwor,Q392143,True,Škwor,Czech rock band,hudební skupina,hudební skupina,[nu metal],NaN,NaN,...,Česko,None,None,None,None,None,None,None,None,NaN
1601,štefan margita,Q8082873,True,Štefan Margita,Slovak opera singer,slovenský operní pěvec,člověk,None,1956.0,NaN,...,Slovensko,[Česko],None,[tenor],[hlas],[medaile Za zásluhy],None,None,None,NaN


In [28]:
americke_jednicky_v_cesku['Žánry'] = americke_jednicky_v_cesku['Interpreti'].apply(lambda x: wikid[wikid['interpret_hlavni'] == x.lower()].explode('zanry')['zanry'].drop_duplicates().to_list())
americke_jednicky_v_cesku['Žánry'] = americke_jednicky_v_cesku['Žánry'].apply(lambda x: ", ".join(x[0:3]))

In [29]:
americke_jednicky_v_cesku

,Interpreti,Nejlepší pozice v ČR,Žánry
0,Maroon 5,2.0,"alternativní rock, funk rock, grunge"
1,Beyoncé,2.0,"soul, pop music, rhythm and blues"
2,Eminem,2.0,"hip hop, hardcore hip hop, Midwest hip hop"
3,Ariana Grande,5.0,"pop music, contemporary R&B, rhythm and blues"
4,Britney Spears,6.0,"pop music, dance-pop, elektropop"
5,Post Malone,7.0,"hip hop, pop rap, pop music"
6,Mariah Carey,11.0,"contemporary R&B, pop music, hip hop"
7,Drake,26.0,"hip hop, contemporary R&B, pop rap"
8,Cardi B,44.0,"hip hop, trap"
9,BTS,NaN,"K-pop, dance-pop, hip hop"


In [30]:
tabulka(americke_jednicky_v_cesku,titulek='V USA 2× nebo víckrát jedničkou, v Česku pod vrcholem', podtitulek='Data z Billboard 100 jsou od roku 2006 do podzimu 2021, česká data do listopadu 2023.', bez_tecky=['Nejlepší pozice v ČR'])

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>V USA 2× nebo víckrát jedničkou, v Česku pod vrcholem</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">Nejlepší pozice v ČR</th><th class="text-nowrap">Žánry</th></tr></thead><tbody class="text-sm"><tr><td>Maroon 5</td><td>2</td><td>alternativní rock, funk rock, grunge</td></tr><tr><td>Beyoncé</td><td>2</td><td>soul, pop music, rhythm and blues</td></tr><tr><td>Eminem</td><td>2</td><td>hip hop, hardcore hip hop, Midwest hip hop</td></tr><tr><td>Ariana Grande</td><td>5</td><td>pop music, contemporary R&amp;B, rhythm and blues</td></tr><tr><td>Britney Spears</td><td>6</td><td>pop music, dance-pop, elektropop</td></tr><tr><td>Post Malone</td><td>7</td><td>hip hop, pop rap, pop music</td></tr><tr><td>Mariah Carey</td><td>11</td><td>contemporary R&amp;B, pop music, hip hop</td></tr><tr><td>Drake</td><td>26</td><td>hip hop, conte

## vs. stream

In [31]:
df = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))
df['week'] = df.apply(lambda x: f"""{x['rok']}-{x['tyden']}""", axis = 1)
df = df[df['rok'] >= 2015]

In [32]:
df['track'] = df['track'].apply(lambda x: nejmensispolecny(x))
df['interpret_hlavni'] = df['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))

In [33]:
stream = pd.read_csv(os.path.join("data","CZ_SINGLES_DIGITAL_TOP_100_tydenni.csv"))
stream = stream[stream['rok'] >= 2015]
stream['week'] = stream.apply(lambda x: f"""{x['rok']}-{x['tyden']}""", axis = 1)

In [34]:
stream_interpreti = stream['interpret_hlavni'].drop_duplicates().to_list()

In [35]:
for s in stream_interpreti:
    malym = nejmensispolecny(s)
    interpreti_slovnik[malym] = s

In [36]:
interpreti = interpreti + stream_interpreti

In [37]:
stream['track'] = stream['track'].apply(lambda x: nejmensispolecny(x))
stream['interpret_hlavni'] = stream['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))

In [38]:
df['w'] = df['week'].rank(method='dense').astype(int)

In [39]:
stream['w'] = stream['week'].rank(method='dense').astype(int)

### Porovnání písní: která bodovala dřív online (-) a která v rádiu (+)

In [40]:
jednicky_radio = df[df['pozice'] == 1].groupby(['interpret_hlavni','track'])['w'].min()

In [41]:
jednicky_stream = stream[stream['pozice'] == 1].groupby(['interpret_hlavni','track'])['w'].min()

In [42]:
jednicky_stream

interpret_hlavni  track         
24kgoldn          mood              287
6ix9ine           gooba             268
adele             easy on me        344
                  hello              39
alan walker       faded              60
                                   ... 
yzomandias        designer flow     297
                  free karlo        399
                  rodinnej typ      355
                  rolls             283
zayn              dusk till dawn    140
Name: w, Length: 96, dtype: int32

In [43]:
(jednicky_stream - jednicky_radio).dropna().sort_values().head()

interpret_hlavni   track            
imagine dragons    bad liar            -54.0
hozier             take me to church   -33.0
twenty one pilots  heathens            -15.0
ed sheeran         i don't care        -13.0
shawn mendes       senorita            -12.0
Name: w, dtype: float64

In [44]:
(jednicky_stream - jednicky_radio).dropna().sort_values().tail()

interpret_hlavni  track              
ed sheeran        shape of you            0.0
alan walker       faded                   0.0
ellie goulding    love me like you do     0.0
ava max           sweet but psycho       11.0
miley cyrus       flowers                21.0
Name: w, dtype: float64

In [45]:
(jednicky_stream - jednicky_radio).dropna().mean()

-8.11111111111111

In [46]:
dodeseti_radio = df[df['pozice'] <= 10].groupby(['interpret_hlavni','track'])['w'].min()
dodeseti_stream = stream[stream['pozice'] <= 10].groupby(['interpret_hlavni','track'])['w'].min()
(dodeseti_stream - dodeseti_radio).dropna().sort_values()

interpret_hlavni  track               
ed sheeran        happier                -87.0
sam smith         too good at goodbyes   -67.0
shawn mendes      stitches               -61.0
justin bieber     sorry                  -58.0
billie eilish     therefore i am         -58.0
                                          ... 
julia michaels    issues                  28.0
alan walker       alone                   31.0
ofenbach          katchi                  38.0
sheppard          geronimo                44.0
david guetta      dangerous               44.0
Name: w, Length: 167, dtype: float64

In [47]:
(dodeseti_stream - dodeseti_radio).dropna().mean()

-9.425149700598803

In [48]:
(dodeseti_stream - dodeseti_radio).dropna().median()

-9.0

In [49]:
dosta_radio = df[df['pozice'] <= 100].groupby(['interpret_hlavni','track'])['w'].min()
dosta_stream = stream[stream['pozice'] <= 100].groupby(['interpret_hlavni','track'])['w'].min()
(dosta_stream - dosta_radio).dropna().sort_values()

interpret_hlavni  track                        
taylor swift      cruel summer                    -204.0
jubel             dancing in the moonlight        -104.0
martin garrix     high on life                     -99.0
ed sheeran        happier                          -69.0
p!nk              just give me a reason            -65.0
                                                   ...  
janek ledecký     sliby se maj plnit o vánocích    259.0
olga lounová      jsem optimista                   314.0
jelen             magdaléna                        364.0
kabát             western boogie                   379.0
amy macdonald     this is the life                 412.0
Name: w, Length: 880, dtype: float64

In [50]:
(dosta_stream - dosta_radio).dropna().mean()

-1.128409090909091

In [51]:
(dosta_stream - dosta_radio).dropna().median()

-3.0

Polovina písniček je o 4 týdny nebo více dřív populární online než v rádiu.

### co lidé stahují a streamují vs. co hrají djové v rádiu

In [52]:
radio = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))

In [53]:
stream['rok'].min()

2015

In [54]:
radio = radio[radio['rok'] >= stream['rok'].min()]

In [55]:
radio['interpret_hlavni'] = radio['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))

#### Porovnání výskytů interpretů v top 100 rádio vs. stream od roku 2014

In [56]:
radio.groupby('interpret_hlavni').size().nlargest(10)

interpret_hlavni
mirai              1195
kryštof             887
marek ztracený      829
chinaski            795
slza                750
ed sheeran          672
xindl x             654
imagine dragons     596
robin schulz        492
jelen               471
dtype: int64

In [57]:
stream.groupby('interpret_hlavni').size().nlargest(10)

interpret_hlavni
viktor sheen       1831
imagine dragons    1386
ed sheeran         1212
calin              1099
yzomandias          979
weeknd              705
nik tendo           695
kygo                623
ektor               539
chainsmokers        527
dtype: int64

In [58]:
pomer = (radio.groupby('interpret_hlavni').size() / stream.groupby('interpret_hlavni').size()).sort_values()
hvezdy_streamu = pd.DataFrame(pomer).reset_index().rename(columns={'interpret_hlavni':'Interpreti',0:'Poměr stream/rádio'}).head(10).replace({"Interpreti": interpreti_slovnik})
hvezdy_streamu['Poměr stream/rádio'] = 1 / hvezdy_streamu['Poměr stream/rádio']
hvezdy_streamu['Poměr stream/rádio'] = hvezdy_streamu['Poměr stream/rádio'].apply(lambda x: f"""{int(x)}×""")
hvezdy_radia = pd.DataFrame(pomer.nlargest(5)).reset_index().rename(columns={'interpret_hlavni':'Interpreti',0:'Poměr rádio/stream'}).replace({"Interpreti": interpreti_slovnik})
hvezdy_radia['Poměr rádio/stream'] = hvezdy_radia['Poměr rádio/stream'].apply(lambda x: f"""{int(x)}×""")

In [59]:
hvezdy_streamu

,Interpreti,Poměr stream/rádio
0,Calin,64×
1,James Hype,60×
2,Echosmith,37×
3,Matoma,37×
4,Rema,32×
5,Passenger,31×
6,Ben Cristovao,28×
7,Rae Sremmurd,25×
8,Kapitán Demo,23×
9,Marc E. Bassy,21×


In [60]:
hvezdy_streamu['Žánry'] = hvezdy_streamu['Interpreti'].apply(lambda x: wikid[wikid['interpret_hlavni'] == x.lower()].explode('zanry')['zanry'].drop_duplicates().to_list())
hvezdy_streamu['Žánry'] = hvezdy_streamu['Žánry'].apply(lambda x: [y for y in x if y != None])
hvezdy_streamu['Žánry'] = hvezdy_streamu['Žánry'].apply(lambda x: ", ".join(x[0:3]))

In [94]:
hvezdy_streamu

,Interpreti,Poměr stream/rádio,Žánry
0,Calin,64×,"alternativní R&B, hip hop"
1,James Hype,60×,"house music, elektronická hudba"
2,Echosmith,37×,"alternativní rock, rock"
3,Matoma,37×,elektronická taneční hudba
4,Rema,32×,Afrobeat
5,Passenger,31×,"folk, indie folk"
6,Ben Cristovao,28×,
7,Rae Sremmurd,25×,hip hop
8,Kapitán Demo,23×,
9,Marc E. Bassy,21×,


In [95]:
hvezdy_streamu['Žánry'] = hvezdy_streamu['Žánry'].apply(lambda x: x if x != "" else "-")

In [96]:
tabulka(hvezdy_streamu, titulek="Komu se výrazně více daří v digitálních než rádiových hitparádách", podtitulek="Srovnání žebříčků top 100 radio a top 100 digital od roku 2015. Číslo vyjadřuje poměr mezi týdny v online a rádiových hitparádách.")

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Komu se výrazně více daří v digitálních než rádiových hitparádách</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">Poměr stream/rádio</th><th class="text-nowrap">Žánry</th></tr></thead><tbody class="text-sm"><tr><td>Calin</td><td>64×</td><td>alternativní R&amp;B, hip hop</td></tr><tr><td>James Hype</td><td>60×</td><td>house music, elektronická hudba</td></tr><tr><td>Echosmith</td><td>37×</td><td>alternativní rock, rock</td></tr><tr><td>Matoma</td><td>37×</td><td>elektronická taneční hudba</td></tr><tr><td>Rema</td><td>32×</td><td>Afrobeat</td></tr><tr><td>Passenger</td><td>31×</td><td>folk, indie folk</td></tr><tr><td>Ben Cristovao</td><td>28×</td><td>-</td></tr><tr><td>Rae Sremmurd</td><td>25×</td><td>hip hop</td></tr><tr><td>Kapitán Demo</td><td>23×</td><td>-</td></tr><tr><td>Marc E. Bassy</td><td>21×</td><td>-</td></tr><

In [63]:
hvezdy_radia

,Interpreti,Poměr rádio/stream
0,Poetika,164×
1,Tomáš Klus,80×
2,Purple Disco Machine,78×
3,Imany,65×
4,David Stypka,63×


In [64]:
hvezdy_radia['Žánry'] = hvezdy_radia['Interpreti'].apply(lambda x: wikid[wikid['interpret_hlavni'] == x.lower()].explode('zanry')['zanry'].drop_duplicates().to_list())
hvezdy_radia['Žánry'] = hvezdy_radia['Žánry'].apply(lambda x: [y for y in x if y != None])
hvezdy_radia['Žánry'] = hvezdy_radia['Žánry'].apply(lambda x: ", ".join(x[0:3]))
hvezdy_radia['Žánry'] = hvezdy_radia['Žánry'].apply(lambda x: "-" if x == "" else x)

In [65]:
tabulka(hvezdy_radia, titulek="Komu se výrazně víc daří v rádiových než digitálních hitparádách", podtitulek="Srovnání žebříčků top 100 radio a top 100 digital od roku 2015. Číslo vyjadřuje poměr mezi týdny v rádiových a online hitparádách.")

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Komu se výrazně víc daří v rádiových než digitálních hitparádách</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">Poměr rádio/stream</th><th class="text-nowrap">Žánry</th></tr></thead><tbody class="text-sm"><tr><td>Poetika</td><td>164×</td><td>-</td></tr><tr><td>Tomáš Klus</td><td>80×</td><td>pop music, folk pop, akustická hudba</td></tr><tr><td>Purple Disco Machine</td><td>78×</td><td>-</td></tr><tr><td>Imany</td><td>65×</td><td>lidová hudba, blues, folk</td></tr><tr><td>David Stypka</td><td>63×</td><td>jazz rock, pop rock</td></tr></tbody><tfoot><tr style="text-align: center;"><td colspan=3>Srovnání žebříčků top 100 radio a top 100 digital od roku 2015. Číslo vyjadřuje poměr mezi týdny v rádiových a online hitparádách.</td></tr></tfoot></table>'

In [66]:
jenomonline = pd.DataFrame(stream[~stream['interpret_hlavni'].isin(radio['interpret_hlavni'].drop_duplicates().to_list())].groupby('interpret_hlavni')['week'].nunique().nlargest(9)).reset_index().rename(columns={'interpret_hlavni':'Interpreti','week':'Týdnů v digi hitparádě'}).replace({"Interpreti": interpreti_slovnik})

In [67]:
jenomonline

,Interpreti,Týdnů v digi hitparádě
0,Wohnout,253
1,Viktor Sheen,251
2,Yzomandias,247
3,Nik Tendo,225
4,Ektor,222
5,Škwor,162
6,Ca$Hanova Bulhar,151
7,Travis Scott,151
8,Earth,150


In [68]:
def zeme(zeme):
    try:
        return zeme[0]
    except:
        return "-"

In [69]:
jenomonline['Země původu'] = jenomonline['Interpreti'].apply(lambda x: zeme(wikid[wikid['interpret_hlavni'] == x.lower()]['zeme_puvodu'].drop_duplicates().to_list()))

In [70]:
def zanry(zanry):
    if len(zanry) == 0:
        return "-"
    if len(zanry) == 1:
        return zanry[0]
    if len(zanry) > 1:
        return zanry[1]

In [71]:
jenomonline['Žánr'] = jenomonline['Interpreti'].apply(lambda x: zanry(wikid[wikid['interpret_hlavni'] == x.lower()].explode('zanry')['zanry'].drop_duplicates().to_list()))

In [72]:
jenomonline = jenomonline.replace({"Ca$Hanova Bulhar": "Ca$hanova Bulhar"})

In [73]:
jenomonline

,Interpreti,Týdnů v digi hitparádě,Země původu,Žánr
0,Wohnout,253,Česko,punk rock
1,Viktor Sheen,251,Kazachstán,hip hop
2,Yzomandias,247,Česko,trap
3,Nik Tendo,225,Česko,trap
4,Ektor,222,Česko,hip hop
5,Škwor,162,Česko,nu metal
6,Ca$hanova Bulhar,151,Česko,trap
7,Travis Scott,151,Spojené státy americké,trap
8,Earth,150,Česko,None


In [74]:
tabulka(jenomonline, titulek="Online bodují, v rádiích zůstali pod stovkou nejhranějších", podtitulek="Data od roku 2015 do roku 2023.")

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Online bodují, v rádiích zůstali pod stovkou nejhranějších</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">Týdnů v digi hitparádě</th><th class="text-nowrap">Země původu</th><th class="text-nowrap">Žánr</th></tr></thead><tbody class="text-sm"><tr><td>Wohnout</td><td>253</td><td>Česko</td><td>punk rock</td></tr><tr><td>Viktor Sheen</td><td>251</td><td>Kazachstán</td><td>hip hop</td></tr><tr><td>Yzomandias</td><td>247</td><td>Česko</td><td>trap</td></tr><tr><td>Nik Tendo</td><td>225</td><td>Česko</td><td>trap</td></tr><tr><td>Ektor</td><td>222</td><td>Česko</td><td>hip hop</td></tr><tr><td>Škwor</td><td>162</td><td>Česko</td><td>nu metal</td></tr><tr><td>Ca$hanova Bulhar</td><td>151</td><td>Česko</td><td>trap</td></tr><tr><td>Travis Scott</td><td>151</td><td>Spojené státy americké</td><td>trap</td></tr><tr><td>Earth</td><td>

## Podíl rapu

In [75]:
wikidzanry = wikid.explode('zanry')

In [76]:
wikidzanry.groupby('zanry')['interpret_hlavni'].nunique().nlargest(60)

zanry
pop music                     346
hip hop                       196
rock                          107
pop rock                      106
contemporary R&B              105
alternativní rock             102
elektronická taneční hudba     64
trap                           63
house music                    61
soul                           53
rhythm and blues               52
dance-pop                      51
elektropop                     51
synthpop                       50
indie pop                      49
pop rap                        44
elektronická hudba             43
hard rock                      43
taneční hudba                  38
electro house                  37
indie rock                     35
country                        33
folk                           29
jazz                           27
heavy metal                    26
electronica                    25
pop punk                       25
Gangsta rap                    24
blues                          24
hardcore

In [77]:
wikidzanry[wikidzanry['zanry'].str.lower().str.contains('hip',na=False)].groupby('zanry')['interpret_hlavni'].nunique().nlargest(60)

zanry
hip hop                   196
hardcore hip hop           24
alternativní hip-hop       22
Southern hip hop           19
East Coast hip hop         13
Atlanta hip hop             9
comedy hip hop              7
hip house                   4
hip-hop kultura             4
Latin hip hop               3
Midwest hip hop             2
conscious hip hop           2
experimentální hip hop      2
Hip-hop soul                2
political hip hop           2
chipmunk soul               1
Christian hip hop           1
industrial hip hop          1
lofi hip hop                1
Name: interpret_hlavni, dtype: int64

In [78]:
wikidzanry[wikidzanry['zanry'].str.lower().str.contains('rap',na=False)].groupby('zanry')['interpret_hlavni'].nunique().nlargest(60)

zanry
trap               63
pop rap            44
Gangsta rap        24
rap                18
rap rock           15
SoundCloud rap     12
emo rap            10
West Coast rap      8
cloud rap           6
mafioso rap         5
Latin trap          5
punk rap            4
progressive rap     4
dirty rap           4
mumble rap          4
EDM trap music      4
UK rap              4
jazz rap            3
rap metal           3
Country rap         2
rage rap            2
Rapcore             2
psychedelic rap     1
rapper              1
trap metal          1
Name: interpret_hlavni, dtype: int64

In [79]:
hiphoperstvo = wikidzanry[(wikidzanry['zanry'].str.lower().str.contains('rap',na=False)) | (wikidzanry['zanry'].str.lower().str.contains('hip',na=False))]['interpret_hlavni'].drop_duplicates().to_list()

In [80]:
hiphoperstvo2 = (wikidzanry[(wikidzanry['zanry'].str.lower().str.contains('rap',na=False)) | (wikidzanry['zanry'].str.lower().str.contains('hip',na=False))].groupby('interpret_hlavni').size() / wikidzanry.groupby('interpret_hlavni').size())
hiphoperstvo2 = hiphoperstvo2[hiphoperstvo2 > 0.66].index.to_list()

***Hiphoper je ten, jehož seznam žánrů obsahuje hiphop/rap alespoň ze 2/3.***

In [81]:
vsichni_s_zanrem = wikidzanry[~wikidzanry['zanry'].isnull()]['interpret_hlavni'].drop_duplicates().to_list()

In [82]:
def podil_hiphoperstva(tablica, jmeno):
    tablica = tablica.copy()
    return pd.Series(tablica[tablica['interpret_hlavni'].isin(hiphoperstvo2)].groupby('rok').size() / tablica[tablica['interpret_hlavni'].isin(vsichni_s_zanrem)].groupby('rok').size(), name=jmeno)

In [83]:
roky_hiphop = [x for x in range(2006,2024)]

In [84]:
cz2 = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))

In [85]:
cz2['interpret_hlavni'] = cz2['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))

In [86]:
cz2[cz2['interpret_hlavni'].isin(hiphoperstvo2)].groupby('interpret_hlavni').size().nlargest(20)

interpret_hlavni
black eyed peas     458
eminem              271
kid laroi           181
flipsyde            163
macklemore          150
jay-z               121
post malone         120
t.i.                108
stromae             106
g-eazy               93
nf                   83
lunchmoney lewis     75
lil nas x            70
50 cent              64
professor green      64
wiz khalifa          63
big ali              61
fort minor           61
mattafix             51
24kgoldn             48
dtype: int64

In [87]:
hiphop_cz = podil_hiphoperstva(cz2, "TOP 100 ČR - rádio").reindex(roky_hiphop)

In [88]:
hiphop_usa = podil_hiphoperstva(b100, "TOP 100 Billboard").reindex(roky_hiphop)

In [89]:
hiphop_stream = podil_hiphoperstva(stream, "TOP 100 ČR - digital").reindex(roky_hiphop)

In [90]:
graf(carovy=[hiphop_cz, hiphop_stream, hiphop_usa], target="hiphop", titulek='Hip hop lidé poslouchají online, v rádiu zní svátečně', podtitulek=f'''Podíl interpretů hrajících hiphop a rap na všech hitparádových příčkách.''', kredity=["Zdroj dat: ČNS IFPI a Wikidata | Vizualizace: Michal Kašpárek, iROZHLAS.cz","https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika"], prvni=True, osay=' ', osay2=' ', skladany=False, max_procenta=50, naopak=False, procenta=[hiphop_cz, hiphop_usa, hiphop_stream], skrytnuly=True)

ModuleNotFoundError: No module named 'highcharts_core'

## Odchylky stream x radio

In [ ]:
sr_pomer = {}
for r in range(2015,2024):
    pracovni_stream = stream[stream['rok'] == r]['interpret_hlavni'].drop_duplicates().to_list()
    pracovni_radio = df[df['rok'] == r]['interpret_hlavni'].drop_duplicates().to_list()
    sr = [x for x in pracovni_stream if x in pracovni_radio]
    sr_pomer[r] = len(sr) / len(set(pracovni_stream + pracovni_radio))

In [ ]:
sr_pomer = pd.Series(sr_pomer, name="Podíl interpretů, kteří se během roku objevili v rádiových i digitálních TOP 100")
sr_pomer

In [ ]:
graf(carovy=[sr_pomer], target="pomer_stream_radio", titulek='Odluka vysílání od streamu', podtitulek=f'''Vkus hudebních dramaturgů a uživatelstva digitálních platforem se posledních devět let rozchází.''', kredity=["Zdroj dat: ČNS IFPI a Wikidata | Vizualizace: Michal Kašpárek, iROZHLAS.cz","https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika"], prvni=True, osay=' ', osay2=' ', max_procenta=50, skladany=False, naopak=False, procenta=[sr_pomer], skrytnuly=True, barvy=["#fd7f6f"])